In [305]:
import numpy as np
import KGInt as kg
import h5py as h5
import matplotlib.pyplot as plt
from IPython.display import clear_output
from functools import reduce
from operator import concat
from ipyparallel import Client

%matplotlib inline

In [5]:
f = h5.File("Levy.h5", "r")

In [15]:
N=2000

In [211]:
limits = []
for i in f["limits"]:
    limits.append(f["limits"][str(i)].value)
alpha_min = f["alphas"]["min_alpha"].value
alpha_max = f["alphas"]["max_alpha"].value
dalpha    = f["alphas"]["dalpha"].value
length    = f["alphas"]["length"].value

In [212]:
alphas = np.linspace(0.5, 1.999, length)
dalpha-(alphas[1]-alphas[0])

0.0

In [276]:
def integrate(x):
    alphas = x[0]
    limits = x[1]
    N = x[2]
    from KGInt import integrateLevyCPU
    import numpy as np
    result = []
    for i in range(len(alphas)):
        rs     = np.linspace(limits[i][0], limits[i][1], N[i], np.double)
        alpha  = np.array([alphas[i]]*N[i], np.double)
        errors     = np.array([1e-8 ]*N[i], np.double)
        error_rngs = np.array([1e-10]*N[i], np.double)
        res    = integrateLevyCPU(rs, alpha, errors, error_rngs)
        result.append( (rs[1]-rs[0],res) )
    return result

In [271]:
length = 6
a = alphas[500:(500+length)]
l = limits[500:(500+length)]

In [258]:
nodes = 6
work = int(length/nodes)

In [259]:
data = list(zip(*[iter(list(zip(a,l,len(a)*[N])))]*work))

invn3 = lambda x,y: (x[0]+[y[0]], x[1]+[y[1]], x[2]+[y[2]])
data = list(map(lambda x: reduce(invn3, x, ([],[],[])), data))

In [260]:
%%bash
sbatch -p hpc2005 -N 6 run_ipengines.sh

Submitted batch job 1614226


In [261]:
%%bash
squeue -u battila93

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1614226   hpc2005 run_ipen battila9 PD       0:00      6 (Resources)
           1614225   hpc2005 run_ipen battila9  R       6:34      6 blade[07-12]


In [262]:
c = Client(profile=open("slurm_profile").readline().replace("\n", ""))
dv = c[:]

In [296]:
c.queue_status()

{'unassigned': 0,
 0: {'completed': 6, 'queue': 1, 'tasks': 0},
 1: {'completed': 6, 'queue': 1, 'tasks': 0},
 2: {'completed': 6, 'queue': 1, 'tasks': 0},
 3: {'completed': 6, 'queue': 1, 'tasks': 0},
 4: {'completed': 6, 'queue': 1, 'tasks': 0},
 5: {'completed': 7, 'queue': 0, 'tasks': 0}}

In [279]:
map_result = dv.map_async(integrate, data)

In [284]:
map_result.status

[None, None, None, None, None, None]

In [285]:
map_result.error

[None, None, None, None, None, None]

In [299]:
map_result.wait_interactive()

   6/6 tasks finished after 1322 s
done


In [300]:
map_result.display_outputs()

[stdout:0] 
->CPU: elapsed time = 141174.0 ms
->CPU: elapsed time = 57165.0 ms
->CPU: elapsed time = 140906.0 ms
->CPU: elapsed time = 140729.0 ms
->CPU: elapsed time = 140684.0 ms
->CPU: elapsed time = 140575.0 ms
->CPU: elapsed time = 140502.0 ms
->CPU: elapsed time = 140389.0 ms
->CPU: elapsed time = 140280.0 ms
->CPU: elapsed time = 140147.0 ms
[stdout:1] 
->CPU: elapsed time = 140902.0 ms
->CPU: elapsed time = 140766.0 ms
->CPU: elapsed time = 40079.0 ms
->CPU: elapsed time = 140462.0 ms
->CPU: elapsed time = 140405.0 ms
->CPU: elapsed time = 140275.0 ms
->CPU: elapsed time = 140220.0 ms
->CPU: elapsed time = 140110.0 ms
->CPU: elapsed time = 140032.0 ms
->CPU: elapsed time = 139970.0 ms
[stdout:2] 
->CPU: elapsed time = 138738.0 ms
->CPU: elapsed time = 138646.0 ms
->CPU: elapsed time = 138586.0 ms
->CPU: elapsed time = 55320.0 ms
->CPU: elapsed time = 138232.0 ms
->CPU: elapsed time = 138146.0 ms
->CPU: elapsed time = 138098.0 ms
->CPU: elapsed time = 138012.0 ms
->CPU: elapsed 

In [302]:
result=map_result.get()

In [307]:
rrr=reduce(concat, result)

In [323]:
f = h5.File("Levy2_new.h5", "w")

In [324]:
def writeList(f, name, data):
    grp = f.create_group(name)
    for i in range(len(data)):
        grp.create_dataset(str(i), data=data[i])

In [377]:
invn2a2 = lambda x,y: (x[0]+[y[0]], x[1]+[y[1][0]], x[2]+[y[1][1]])
prepdat = reduce(lambda x,y:(x[0]+[y[0]],x[1]+[y[1]]), map(lambda x:(x[0],x[1][:,0]), rrr), ([],[]))

In [385]:
limits = l
alphas = a

In [387]:
writeList(f, "dr", prepdat[0])
writeList(f, "Lr", prepdat[1])
writeList(f, "limits", limits)
grp = f.create_group("/alphas")
grp.create_dataset("dalpha", data=np.array(alphas[1]-alphas[0], np.double))
grp.create_dataset("min_alpha", data=np.array(alphas[0], np.double))
grp.create_dataset("max_alpha", data=np.array(alphas[-1], np.double))
grp.create_dataset("length", data=np.array(len(alphas), np.int))

<HDF5 dataset "length": shape (), type "<i8">

In [395]:
f.close()